# Geração de instâncias

In [1]:
import math
import json

In [2]:
# Salva um arquivo JSON com o dicionário fornecido
def imprimir_json(nome_arquivo, dados):
    with open(nome_arquivo, 'w', encoding='utf-8') as arquivo:
        json.dump(dados, arquivo, ensure_ascii = False, indent = 4)

In [3]:
# Número de vértices possíveis
V = [100, 150, 200, 250]

for v in V:
    coordenadas = []
    
    K = [0, int(v/2), v]
    
    with open('mo824_atividade2_coords','r') as coordsFile:
        for i in range(v):
            line = next(coordsFile)
            coordenadas.append([int(x) for x in line.split()])
            
    custos = {str((i, j)): [
        math.sqrt(sum((coordenadas[i][k]-coordenadas[j][k])**2 for k in range(2))),
        math.sqrt(sum((coordenadas[i][k+2]-coordenadas[j][k+2])**2 for k in range(2)))]
        for i in range(v) for j in range(v)}
    
    for k in K:
        instancia = {}
        instancia["k"] = k
        instancia["custos"] = custos
        instancia["V"] = v
        
        imprimir_json(f'instancia-vertices-{v}-similaridade-{k}.json', instancia)